In [7]:
%matplotlib inline

## Written by Gaojie Jin and updated by Xiaowei Huang, 2021
##
## For a 2-nd year undergraduate student competition on 
## the robustness of deep neural networks, where a student 
## needs to develop 
## 1. an attack algorithm, and 
## 2. an adversarial training algorithm
##
## The score is based on both algorithms. 
###########################################################################

In [8]:

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.autograd import Variable
import argparse
import time
import copy

# Change this id to your student id
id_ = 1000

# setup training parameters
parser = argparse.ArgumentParser(description='PyTorch MNIST Training')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--test-batch-size', type=int, default=128, metavar='N',
                    help='input batch size for testing (default: 128)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')


args = parser.parse_args(args=[]) 

# judge cuda is available or not
use_cuda = not args.no_cuda and torch.cuda.is_available()
#device = torch.device("cuda" if use_cuda else "cpu")
device = torch.device("cpu")

torch.manual_seed(args.seed)
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

###############    don't change the below code    #####################
###########################################################################

In [9]:
train_set = torchvision.datasets.FashionMNIST(root='data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True)

test_set = torchvision.datasets.FashionMNIST(root='data', train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))
test_loader = DataLoader(test_set, batch_size=args.batch_size, shuffle=True)

# define fully connected network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        output = F.log_softmax(x, dim=1)
        return output

############    end of "don't change the below code"   ######################
#############################################################################

In [10]:
#generate adversarial data, you can define your adversarial method
def adv_attack(model, X, y, device):
    X_adv = Variable(X.data)
    
    ################################################################################################
    ## Note: below is the place you need to edit to implement your own attack algorithm
    ################################################################################################
    
    random_noise = torch.FloatTensor(*X_adv.shape).uniform_(-0.1, 0.1).to(device)
    X_adv = Variable(X_adv.data + random_noise)
    
    ################################################################################################
    ## end of attack method
    ################################################################################################
    
    return X_adv

#train function, you can use adversarial training
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0),28*28)
        
        #use adverserial data to train the defense model
        #adv_data = adv_attack(model, data, target, device=device)
        
        #clear gradients
        optimizer.zero_grad()
        
        #compute loss
        #loss = F.nll_loss(model(adv_data), target)
        loss = F.nll_loss(model(data), target)
        
        #get gradients and update
        loss.backward()
        optimizer.step()
        


#predict function
def eval_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.view(data.size(0),28*28)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = correct / len(test_loader.dataset)
    return test_loss, test_accuracy

def eval_adv_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0),28*28)
        adv_data = adv_attack(model, data, target, device=device)
        with torch.no_grad():
            output = model(adv_data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = correct / len(test_loader.dataset)
    return test_loss, test_accuracy

#main function, train the dataset and print train loss, test loss for each epoch
def train_model():
    model = Net().to(device)
    
    ################################################################################################
    ## Note: below is the place you need to edit to implement your own training algorithm
    ##       You can also edit the functions such as train(...). 
    ################################################################################################
    
    optimizer = optim.SGD(model.parameters(), lr=args.lr)
    for epoch in range(1, args.epochs + 1):
        start_time = time.time()
        
        #training
        train(args, model, device, train_loader, optimizer, epoch)
        
        #get trnloss and testloss
        trnloss, trnacc = eval_test(model, device, train_loader)
        advloss, advacc = eval_adv_test(model, device, train_loader)
        
        #print trnloss and testloss
        print('Epoch '+str(epoch)+': '+str(int(time.time()-start_time))+'s', end=', ')
        print('trn_loss: {:.4f}, trn_acc: {:.2f}%'.format(trnloss, 100. * trnacc), end=', ')
        print('adv_loss: {:.4f}, adv_acc: {:.2f}%'.format(advloss, 100. * advacc))

    ################################################################################################
    ## end of training method
    ################################################################################################

    print('\n' + '='*80)
    print('Training completed! Model saved as: {}.pt'.format(id_))
    print('='*80)
    print('\nTo evaluate your attack and defence scores:')
    print('  1. Test your attack:  python Evaluate_attack.py')
    print('  2. Test your defence: python Evaluate_defence.py')
    print('\nMake sure to:')
    print('  - Update attack method in Evaluate_attack.py')
    print('  - Update model file path in Evaluate_defence.py')
    print('  - Verify epsilon constraint < 0.11 (see p_distance output below)')
    print('='*80 + '\n')
    
    #save the model
    torch.save(model.state_dict(), str(id_)+'.pt')
    return model

#compute perturbation distance
def p_distance(model, train_loader, device):
    p = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0),28*28)
        data_ = copy.deepcopy(data.data)
        adv_data = adv_attack(model, data, target, device=device)
        p.append(torch.norm(data_-adv_data, float('inf')))
    print('epsilon p: ',max(p))


# Note: below is for testing/debugging purpose, please comment them out in the submission file
###############################################################################################

In [11]:
#Comment out the following command when you do not want to re-train the model
#In that case, it will load a pre-trained model you saved in train_model()
model = train_model()

#Call adv_attack() method on a pre-trained model'
#the robustness of the model is evaluated against the infinite-norm distance measure
#important: MAKE SURE the infinite-norm distance (epsilon p) less than 0.11 !!!
p_distance(model, train_loader, device)

D:\Software\anaconda3\envs\aisafety\Lib\site-packages\torch\nn\_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch 1: 10s, trn_loss: 1.9006, trn_acc: 38.39%, adv_loss: 1.9011, adv_acc: 38.39%
Epoch 2: 11s, trn_loss: 1.0741, trn_acc: 61.13%, adv_loss: 1.0752, adv_acc: 61.16%
Epoch 3: 10s, trn_loss: 0.8683, trn_acc: 68.08%, adv_loss: 0.8693, adv_acc: 68.05%
Epoch 4: 10s, trn_loss: 0.7805, trn_acc: 69.71%, adv_loss: 0.7822, adv_acc: 69.67%
Epoch 5: 10s, trn_loss: 0.7016, trn_acc: 74.82%, adv_loss: 0.7034, adv_acc: 74.76%
Epoch 6: 10s, trn_loss: 0.6594, trn_acc: 76.19%, adv_loss: 0.6614, adv_acc: 76.08%
Epoch 7: 10s, trn_loss: 0.6244, trn_acc: 77.27%, adv_loss: 0.6271, adv_acc: 77.13%
Epoch 8: 10s, trn_loss: 0.5785, trn_acc: 79.36%, adv_loss: 0.5820, adv_acc: 79.27%
Epoch 9: 11s, trn_loss: 0.5520, trn_acc: 80.75%, adv_loss: 0.5555, adv_acc: 80.61%
Epoch 10: 12s, trn_loss: 0.5375, trn_acc: 81.05%, adv_loss: 0.5409, adv_acc: 80.87%

Training completed! Model saved as: 1000.pt

To evaluate your attack and defence scores:
  1. Test your attack:  python Evaluate_attack.py
  2. Test your defence: pytho